In [380]:
import matplotlib as mpl
import matplotlib.cm
import matplotlib.colors
import numpy as np
from math import floor

In [381]:
import json
import os

In [382]:
with open('test_fraud.json') as f:
    data = json.load(f)

zip_codes = data['zip_codes']
max_fraud = float(data['max_fraud_pct']) / 100
n_todraw = len(data['result'])

In [383]:
center = [37.8, -96.9]
zoom = 4

m = Map(center=center, zoom=zoom)

n_colors = 10

In [384]:
def make_colors(colormap=mpl.cm.PuOr):
    data = np.linspace(0.0,1.0,n_colors)
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(data)]
    return c

In [385]:
colors = make_colors()

In [386]:
to_plot = {}

for r in data['result']:
    fraud_pct = float(r['fraud_pct']) / 100
    color = colors[int(floor((float(n_todraw) / n_colors) * (float(fraud_pct) / float(max_fraud))))]
    
    r['properties'] = {'style': {'color':color, 'weight': 1, 'fillColor':color, 'fillOpacity':0.5}, 'fraud_pct': fraud_pct}
    
    to_plot[r["zip_code"]] = r

In [423]:
plot_list = []

In [424]:
for f in os.listdir("./json-zips/zip_codes_for_the_usa"):
    with open(os.path.join("./json-zips/zip_codes_for_the_usa", f)) as fdata:
        area = json.load(fdata)
        map_out(area)

In [425]:
def map_out(area):
    for feature in area['features']:
        zip_code = zip_fix(feature['properties']['ZIP'])
        if zip_code in zip_codes:
            draw_data(zip_code, feature)

In [426]:
def zip_fix(zip_code):
    zip_code = str(zip_code)
    while len(zip_code) < 5:
        zip_code = "0" + zip_code
    return zip_code

In [427]:
def draw_data(zip_code, feature):
    for k, v in to_plot[zip_code]['properties'].iteritems():
        feature['properties'][k] = v
    plot_list.append(feature)

In [428]:
data['features'] = plot_list

In [430]:
g = GeoJSON(data=data)
m.add_layer(g)

In [431]:
m

In [432]:
zoom_slider = IntSlider(description='Zoom', min=2, max=15, value=zoom)
jslink((zoom_slider, 'value'), (m, 'zoom'))
zoom_slider